<a href="https://colab.research.google.com/github/sahankrt20/Brain-tumor-segmentation-/blob/main/brain_tumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import seaborn as sns
from PIL import Image
from torchvision import transforms,models
from torch.utils.data import DataLoader
import torch
import cv2
from tqdm import tqdm
import glob
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [43]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [21]:
train_dir='/content/drive/MyDrive/brain tumor/Training'
test_dir='/content/drive/MyDrive/brain tumor/Testing'

In [45]:
class_names=["glioma","meningioma","notumor","pituitary"]
train_sizes=[
    len(os.listdir("/content/drive/MyDrive/brain tumor/Training/glioma")),
    len(os.listdir("/content/drive/MyDrive/brain tumor/Training/meningioma")),
    len(os.listdir("/content/drive/MyDrive/brain tumor/Training/notumor")),
    len(os.listdir("/content/drive/MyDrive/brain tumor/Training/pituitary"))
]
test_sizes=[
    len(os.listdir("/content/drive/MyDrive/brain tumor/Testing/glioma")),
    len(os.listdir("/content/drive/MyDrive/brain tumor/Testing/meningioma")),
    len(os.listdir("/content/drive/MyDrive/brain tumor/Testing/notumor")),
    len(os.listdir("/content/drive/MyDrive/brain tumor/Testing/pituitary"))
]


In [22]:
train_paths=[]
train_labels=[]
for label in os.listdir(train_dir):
  for image in os.listdir(os.path.join(train_dir,label)):
    train_paths.append(os.path.join(train_dir,label,image))
    train_labels.append(label)

In [6]:
train_paths

['/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0000.jpg',
 '/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0006.jpg',
 '/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0004.jpg',
 '/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0002.jpg',
 '/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0009.jpg',
 '/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0005.jpg',
 '/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0007.jpg',
 '/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0008.jpg',
 '/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0003.jpg',
 '/content/drive/MyDrive/brain tumor/Training/pituitary/Tr-piTr_0001.jpg']

In [11]:


test_df = '/content/drive/MyDrive/brain tumor/Testing'
file_paths = []
labels = []

folds = os.listdir(test_df)  # Getting the list of subdirectories (categories)

for fold in folds:
    fold_path = os.path.join(test_df, fold)  # Full path of each category folder
    files = os.listdir(fold_path)  # Corrected: List files in the category folder

    for file in tqdm(files):
        file_path = os.path.join(fold_path, file)
        file_paths.append(file_path)
        labels.append(fold)  # Assigning correct label


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


# READING IMAGES


In [ ]:
tumor=[]
healthy=[]
for f in glob.iglob(""):
  img=cv2.imread(f)
  img=cv2.resize(img,(224,224))
  b,g,r=cv2.split(img)
  img=cv2.merge([r,g,b])
  tumor.append(img)
for f in glob.iglob(""):
  img=cv2.imread(f)
  img=cv2.resize(img,(224,224))
  b,g,r=cv2.split(img)
  img=cv2.merge([r,g,b])
  healthy.append(img)



In [ ]:
health=np.array(healthy)
not_healthy=np.array(tumor)
both=np.concatenate((health,not_healthy),axis=0)

In [12]:
len(train_paths)

10

In [13]:
len(train_labels)

10

In [14]:
model=models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 180MB/s]


In [15]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [17]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [18]:
transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [ ]:
class Brain_tumor(nn.Module):
  def __init__(self) :
     super().__init__()
     self.transform=transform
     self.classes=['glioma', 'meningioma', 'notumor', 'pituitary']

